# Deep 3
### Matan Leventer 
### Rom Amsili 

In [ ]:
!pip install pretty_midi

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from gensim.models import Word2Vec
from gensim.models.word2vec import Text8Corpus
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_regression, make_classification
from torch import nn
from torch.nn import functional as F
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from google.colab import drive
import os
import cv2
from torchvision import transforms
from PIL import Image
import re
from collections import defaultdict
from sklearn.model_selection import train_test_split
import pretty_midi
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import random

In [ ]:
!rm -rf runs/*
if not os.path.exists('./text8'):
    !wget http://mattmahoney.net/dc/text8.zip
    !unzip text8.zip

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
drive.mount('/content/drive/')

In [ ]:
word2vec_size = 300
batch_size = 128

In [ ]:
corpus =Text8Corpus('./text8')
word_model = Word2Vec(sentences=corpus, size=word2vec_size, window=5, min_count=1, workers=8)
word_model.save("/content/drive/MyDrive/Deep_models/word2vec.model")

In [ ]:
from gensim.models import KeyedVectors

word_model = KeyedVectors.load("/content/drive/MyDrive/Deep_models/word2vec.model", mmap='r')

#Melody 

In [ ]:
def get_midi_file_instrument_data(index_word, tpd, midi_file):
    #  Features we want to extract:
    start_time = index_word * tpd
    end_time = start_time + tpd
    matrix_chroma = midi_file.get_chroma(times = np.array([start_time,end_time])).mean(axis=1)
    avarage_velocity= avarage_pitch=num_of_instruments= num_of_notes= has_drums = 0
    for instrument in midi_file.instruments:
        range = False  
        for note in instrument.notes:
            if start_time <= note.start:
                if note.end <= end_time:
                    if instrument.is_drum:
                      has_drums = 1
                    else:
                      has_drums=0
                    range = True
                    num_of_notes += 1
                    avarage_pitch = avarage_pitch + note.pitch
                    avarage_velocity = avarage_velocity + note.velocity
                else:  
                    break
        if range:
          num_of_instruments += 1

    if num_of_notes > 0:  
        avarage_velocity = avarage_velocity / num_of_notes
        avarage_pitch = avarage_pitch / num_of_notes
        
    final_features = np.concatenate((np.array([avarage_velocity, avarage_pitch, num_of_instruments,0,0, has_drums]),  matrix_chroma), axis=0) 
    return final_features

In [ ]:
def get_melody_path(artist,name):
  output = ''
  for word in artist.split(' '):
      output += word + '_'
  output += '-'
  for word in name.split(' '):
    output += word + '_'
  return output[:-1]+'.mid'

def prepare_text(df_path):
  df1 = pd.read_csv(df_path,header=None)
  df = df1.iloc[:,:3]
  df.columns = ['Artist','Name','Lyrics']
  df['Lyrics']= df['Lyrics'].apply(lambda x:' '.join(re.sub(r'[^\w\s]', '', x).split()))
  text = df['Lyrics']
  X = []
  y = []
  for j,row in df.iterrows():
    lyrics = row['Lyrics'].split(' ')
    midi_path = f"/content/drive/MyDrive/midi_files/{get_melody_path(row['Artist'],row['Name'])}"
    try:
      midi_data = pretty_midi.PrettyMIDI(midi_path)
    except Exception as e:
      continue
    end_time = midi_data.get_end_time()
    avg_time_per_word = end_time/len(lyrics)
    beat_freq = np.mean([x - midi_data.get_beats()[i - 1] for i, x in enumerate(midi_data.get_beats())][1:])
    len_instruments = len(midi_data.instruments)

    for i in range(len(lyrics)-1):
      word_melody = get_midi_file_instrument_data(i,avg_time_per_word,midi_data)
      word_melody[3] = len_instruments
      word_melody[4] = beat_freq
      if word_model.wv.__contains__(lyrics[i]) and word_model.wv.__contains__(lyrics[i+1]):
        X.append(torch.cat((torch.tensor(word_model.wv[lyrics[i]]),
                            torch.from_numpy(word_melody).float()),0))
        y.append(torch.tensor(word_model.wv[lyrics[i+1]]))
  return X,y

def prepare_test(df_path):
  df1 = pd.read_csv(df_path,header=None)
  df = df1.iloc[:,:3]
  df.columns = ['Artist','Name','Lyrics']
  df['Lyrics']= df['Lyrics'].apply(lambda x:' '.join(re.sub(r'[^\w\s]', '', x).split()))
  text = df['Lyrics']
  X = [[],[],[],[],[]]
  y = []
  for j,row in df.iterrows():
    lyrics = row['Lyrics'].split(' ')
    midi_path = f"/content/drive/MyDrive/midi_files/{get_melody_path(row['Artist'],row['Name'])}"
    try:
      midi_data = pretty_midi.PrettyMIDI(midi_path)
    except Exception as e:
      continue
    end_time = midi_data.get_end_time()
    avg_time_per_word = end_time/len(lyrics)
    beat_freq = np.mean([x - midi_data.get_beats()[i - 1] for i, x in enumerate(midi_data.get_beats())][1:])
    len_instruments = len(midi_data.instruments)
    for i in range(len(lyrics)-1):
      word_melody = get_midi_file_instrument_data(i,avg_time_per_word,midi_data)
      word_melody[3] = len_instruments
      word_melody[4] = beat_freq
      X[j].append(torch.from_numpy(word_melody).float())
  return X

# X,y = prepare_text('lyrics_train_set.csv')


In [ ]:
len(X)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=67)

In [ ]:
len(X_val)

In [ ]:
train_data = [[X_train[i], y_train[i]] for i in range(len(X_train))]
val_data = [[X_val[i], y_val[i]] for i in range(len(X_val))]

trainloader = torch.utils.data.DataLoader(train_data, shuffle=False, batch_size=batch_size)
valloader = torch.utils.data.DataLoader(val_data, shuffle=False, batch_size=batch_size)

In [ ]:
# Define the model architecture
class LstmModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size,dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out


class GruModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size,dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out)
        return out

class LstmModel_fc(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size,dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.fc1 = nn.Linear(18, 100)

    def forward(self, x):

        out_m = self.fc1(x[:,300:])
        out = torch.cat((x[:,:300],out_m),1)
        out, _ = self.lstm(out)
        out = self.fc2(out)
        return out


In [ ]:
def train(dataloader, model, loss_fn, optimizer,name):
    running_loss = 0.0
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)

        # Compute prediction error
        pred = model(X.float())
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch % 50 == 0 and batch!=0:
            loss, current = loss.item(), batch * batch_size
            writer.add_scalar(f'training/loss_{name}',
                              running_loss / 50,
                              epoch * len(dataloader) + batch)
            running_loss =0.0
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def sample(top10_dict):
  top10_values = softmax(list(top10_dict.values()))
  inx = np.argmax(np.random.multinomial(1,top10_values,1))
  return list(top10_dict)[inx]

def test(dataloader, model, loss_fn,name,topn):
    running_loss = 0.0
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss = []
    correct = 0
    q=10
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X = X.to(device)
            y = y.to(device)
            batch_pred = model(X.float())
            loss = loss_fn(batch_pred, y)
            test_loss.append(loss)
            if q==0:
              running_loss += loss.item()
              writer.add_scalar(f'validation/loss_{name}',
                                running_loss / 100,
                                epoch * len(dataloader) + batch)
              running_loss =0.0
              q=10
            q-=1
            sample = 100
            for i,pred in enumerate(batch_pred):
              if i%sample == 0:
                top10 = dict(word_model.most_similar(positive=[np.array(pred.cpu())], topn=topn))
                true_word = word_model.wv.most_similar(positive=[np.array(y[i].cpu())], topn=1)[0][0]
                if true_word in top10:
                  correct+=1   #top10[y]
                test_loss.append(loss_fn(pred, y))

    test_loss = sum(test_loss)/len(test_loss)
    size/=sample
    correct /= size
    print(f"Accuracy: {(correct*100):>0.1f}%, Avg loss: {test_loss:>7f} \n")
    return correct,test_loss

In [ ]:
writer = SummaryWriter()

In [ ]:
! pip install tensorboard

In [ ]:
%load_ext tensorboard

# Expirement with 2 architectures

In [ ]:
batch_size = 128
model_1 = LstmModel(input_size=word2vec_size+18, hidden_size=512, num_layers=2, output_size=word2vec_size,dropout=0).to(device)
model_2 = LstmModel_fc(input_size=word2vec_size+100, hidden_size=512, num_layers=2, output_size=word2vec_size,dropout=0).to(device)

In [ ]:
# Define the loss function and optimizer
models = ["model_1","model_2"]
loss_list = []

for i,model in enumerate([model_1,model_2]):
  print(models[i])
  validation_loss = 100.0
  loss_fn = torch.nn.L1Loss()  
  optimizer = optim.Adam(model.parameters(), lr = 0.001)
  # Train the model
  for epoch in range(4):
      print(epoch)
      # hidden = model.init_hidden(batch_size=1)
      train(trainloader, model, loss_fn, optimizer,models[i])
      acc,loss = test(valloader, model, loss_fn,models[i])
      print('losses',validation_loss - loss,loss)
      if validation_loss - loss < 0.000001:
        print("Loss convergence")
        break
      validation_loss  = loss
      loss_list.append(loss)


In [ ]:
acc,loss = test(valloader, model_2, loss_fn,"model_2")


#Genreate

In [ ]:
model_1= torch.load('/content/drive/MyDrive/Deep3/model_1')
model_2= torch.load('/content/drive/MyDrive/Deep3/model_2')

In [ ]:
# Upload test_set.csv!
X_test,y_test = prepare_text('lyrics_test_set.csv')
X_melody = prepare_test('lyrics_test_set.csv')

In [ ]:
def count_words_in_song(path='lyrics_test_set.csv'):
  df1 = pd.read_csv(path,header=None)
  df = df1.iloc[:,:3]
  df.columns = ['Artist','Name','Lyrics']
  df['Lyrics']= df['Lyrics'].apply(lambda x:' '.join(re.sub(r'[^\w\s]', '', x).split()))
  text = df['Lyrics']
  len_lyrics = {}
  for j,row in df.iterrows():
    lyrics = row['Lyrics'].split(' ')
    len_lyrics[j] = 0
    for i in range(len(lyrics)-1):
      if word_model.wv.__contains__(lyrics[i]) and word_model.wv.__contains__(lyrics[i+1]):
        len_lyrics[j] += 1
  return len_lyrics
count_words_in_song()

In [ ]:
model = model_1
N = 298
m=0
# first_word = word_model.wv.most_similar(positive=[X_test[N][:300].cpu().detach().numpy().squeeze()], topn=1)[0][0]
first_word = word_model.wv['deep']
next = "\x1B[4m"+'Verse 1:\n'+"\x1B[0m"+ 'deep' + " "
gen = torch.cat((torch.tensor(first_word).to(device),X_melody[m][0].to(device)),0)
new_line=random.randint(3, 6)
lines=0
Chorus = ['',False]
for i in range(150):
  new_line-=1
  if new_line==0:
    new_line=random.randint(3, 6)
    next+='\n'
    if Chorus[1]:
      Chorus[0] += '\n'
    lines +=1
    if lines == 6:
      next+="\x1B[4m"+'\nCourse:\n'+"\x1B[0m" 
      Chorus[1] = True
    if lines == 9:
      next+="\x1B[4m"+'\nVerse 2:\n'+"\x1B[0m" 
      Chorus[1] = False
    if lines == 15:
      next+="\x1B[4m"+'\nChorus:\n'+"\x1B[0m"+Chorus[0] 
      break
  word = model(gen[None].to(device))
  gen = torch.cat((word.squeeze(),X_melody[m][i+1].to(device)),0)
  if Chorus[1]:
    word = sample(dict(word_model.wv.most_similar(positive=[word.cpu().detach().numpy().squeeze()], topn=10))) + ' '
    next += word
    Chorus[0] += word 
  else:
    next += sample(dict(word_model.wv.most_similar(positive=[word.cpu().detach().numpy().squeeze()], topn=10))) + ' '
print(next)

# Model Comparison
## expirement with different hyperparameters

In [ ]:
list_model= ['Lstm','Gru']
list_batch_size = [64,128]
list_dropout = [0 ,0.2]
list_hidden_size = [256,512]
list_num_layers = [2 ,8]

In [ ]:
acc_dict = {}
loss_dict = {}
for m in list_model:
  for hidden in list_hidden_size:
    for num_layers in list_num_layers:
      for dropout in list_dropout:
        for batch_size in list_batch_size:
          validation_loss = 100.0
          if m == 'Lstm':
            model = LstmModel(input_size=word2vec_size+18, hidden_size=hidden, num_layers=num_layers, output_size=word2vec_size,dropout=dropout).to(device)
          else:
            model = GruModel(input_size=word2vec_size+18, hidden_size=hidden, num_layers=num_layers, output_size=word2vec_size,dropout=dropout).to(device)
          trainloader = torch.utils.data.DataLoader(train_data, shuffle=False, batch_size=batch_size)
          valloader = torch.utils.data.DataLoader(val_data, shuffle=False, batch_size=batch_size)
          loss_fn = torch.nn.L1Loss()  
          optimizer = optim.Adam(model.parameters(), lr = 0.001)
          name = f'{m}_{hidden}_{num_layers}_{dropout}_{batch_size}'
          loss_list = []
          for epoch in range(4):
              train(trainloader, model, loss_fn, optimizer,name)
              acc,loss = test(valloader, model, loss_fn,name)
              if validation_loss - loss <0.01:
                print("Loss convergence")
                break
              validation_loss  = loss
              loss_list.append(loss)
          loss_dict[f'{m}_{hidden}_{num_layers}_{dropout}_{batch_size}'] = loss_list
          torch.save(model,f'/content/drive/MyDrive/Deep3/{name}')
          print(f'{name} finished')

In [ ]:
dict_acc = {}
for m in list_model:
  for hidden in list_hidden_size:
    for num_layers in list_num_layers:
      for dropout in list_dropout:
        for batch_size in list_batch_size:
            name = f'{m}_{hidden}_{num_layers}_{dropout}_{batch_size}'
            model= torch.load(f'/content/drive/MyDrive/Deep3/{name}')
            print(name)
            acc,loss = test(valloader, model, loss_fn,name)
            dict_acc[name] = acc


In [ ]:
sorted_by_value = dict(sorted(dict_acc.items(), key=lambda item: item[1],reverse=True))
for i in sorted_by_value.items():
  print(i)


In [ ]:
for i,(k,v) in enumerate(sorted_by_value.items()):
  model = k.split('_')[0]
  hidden_size = k.split('_')[1]
  num_lay = k.split('_')[2]
  dropout = k.split('_')[3]
  batch_size = k.split('_')[4]
  print(f'model: {model}, hidden_size: {hidden_size}, num layers: {num_lay}, dropout: {dropout}, batch_size: {batch_size}\n\
The accuracy is: {v}\n\n')
    

In [ ]:
%tensorboard --logdir runs